<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Train-from-specified-config-files" data-toc-modified-id="Train-from-specified-config-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Train from specified config files</a></span><ul class="toc-item"><li><span><a href="#Build-model" data-toc-modified-id="Build-model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Build model</a></span></li><li><span><a href="#Augment-data" data-toc-modified-id="Augment-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Augment data</a></span></li><li><span><a href="#Auxiliary-train-time-functions" data-toc-modified-id="Auxiliary-train-time-functions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Auxiliary train time functions</a></span></li><li><span><a href="#Training-functions" data-toc-modified-id="Training-functions-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Training functions</a></span></li><li><span><a href="#Unifying-Function" data-toc-modified-id="Unifying-Function-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Unifying Function</a></span></li><li><span><a href="#Run-training" data-toc-modified-id="Run-training-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Run training</a></span></li></ul></li></ul></div>

# Train from specified config files
       -- built for FF+ dataset with file structure as required by Keras' flow_from_directory method

In [6]:
# See available GPU RAM 
# !nvidia-smi # can also be run from linux shell while GPU is training
# !nvidia-smi dmon # this will stream memory utilisation
# !watch -n0.1 nvidia-smi # better way to see GPU utilisation
# !htop # cpu threads and if they're all working
# !pip3 install --no-cache-dir -I tensorflow==2.2 # use if no gpu is attached so code will run 
# !sudo kill -9 PID # clear GPU memory where 9 is PID number

In [7]:
# # Augmentation installations
# !pip install CMake
# !pip install dlib
# !pip install face_recognition

In [8]:
# This cell has the latest set up for AI Platform

from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import glob
import cv2
from io import BytesIO
from PIL import Image
from numpy import expand_dims
import tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import *
import warnings
import logging
from IPython.display import clear_output
from collections import Counter
import pickle
import json
import sys
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/augmentations')
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/hyperparameters')
sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/visualisations')
import VisualisationTools as plotting
import hyper_utils as hp

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(100)
plot_losses = plotting.PlotLearning()
os.chdir('/home/jupyter/DeepFake-2019-20')

# # Augmentation libraries
# import face_recognition
# import cutout_augmentation as ca

print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.1.1-dlenv_tfe


In [9]:
# Specify config file
config_number=0
config_file='config{}'.format(config_number)
def obtain_configs(number):
    '''Extracts hyperparameters from config file given the config file number.'''
    with open('./configs/config{}.json'.format(number)) as f:
        params = json.load(f)

    return params

params = obtain_configs(config_number)
params

{'architecture': 'efficientnet',
 'epochs': 140,
 'batch_size': 256,
 'learning_rate_type': 'cosine_decay',
 'learning_rate': 0.0006547675102413338,
 'patience': 6,
 'weight_initialisation': 'imagenet',
 'optimiser': 'adam',
 'momentum': 0.1,
 'nesterov': True,
 'label_smoothing': 0.04,
 'dropout': 0.0,
 'target_size': 224,
 'class_weights': True,
 'warmup_epochs': 7}

## Build model

In [10]:
def build_model(dropout, lr_rate, architecture, frozen_base=True):
    '''Builds a specified network with the selected dropout after the last dense layer.

    Architectures that can be selected are:
    vgg, xception, resnet50, mobilenet, efficientnet, densenet
    
    Optimiser is Adam, with a provided learning rate (lr_rate) and fixed
    decay 1e-6, loss is traditionally categorical_crossentropy.'''

    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

    if architecture=='xception':
        from tensorflow.keras.applications.xception import Xception
        conv_base = Xception(weights='imagenet', include_top=False,
                        input_shape=(224,224,3))

    elif architecture=='vgg':
        from tensorflow.keras.applications.vgg16 import VGG16
        conv_base = VGG16(weights='imagenet', include_top=False,
                        input_shape=(224,224,3))
      
    elif architecture=='resnet50':
        from tensorflow.keras.applications.resnet50 import ResNet50
        conv_base = ResNet50(weights='imagenet', include_top=False,
                        input_shape=(224,224,3))
      
    elif architecture=='mobilenet':
        from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
        conv_base = MobileNetV2(weights='imagenet', include_top=False,
                        input_shape=(224,224,3))

    elif architecture== 'efficientnet':
        # EfficientNetB7 has the highest top-1 accuracy on imagenet
        # among EfficientNextB{0:7}
        !pip install git+https://github.com/qubvel/efficientnet
        from efficientnet.tfkeras import EfficientNetB0
        conv_base = EfficientNetB0(weights='noisy-student', include_top=False,
                        input_shape=(224,224,3))
        
    elif architecture== 'densenet':
        from tensorflow.keras.applications.densenet import DenseNet201
        conv_base = DenseNet201(weights='imagenet', include_top=False,
                        input_shape=(224,224,3))

    elif architecture not in ['vgg', 'xception', 'resnet50',
                              'mobilenet', 'efficientnet', 'densenet']:
        return "An unknown network is specified"
    

    outputconv_base = conv_base.output
    t_flat = Flatten()(outputconv_base)
    t_dense1 = Dense(1024, activation='relu')(t_flat)
    t_dense2 = Dense(256, activation='relu')(t_dense1)
    t_dense3 = Dense(128, activation='relu')(t_dense2)
    t_do = Dropout(dropout)(t_dense3)
    predictions = Dense(2, activation= 'softmax')(t_do)

    model = Model(inputs=conv_base.input, outputs=predictions, name = 'model')
    
    if frozen_base:
        conv_base.trainable = False # freeze the convolutional base
        
    else: 
        conv_base.trainable = True


    if params['optimiser']=='adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr_rate)
    
    elif params['optimiser']=='sgd' and params['nesterov']=='True':
        opt = tf.keras.optimizers.SGD(learning_rate=lr_rate, momentum=params['momentum'], nesterov=True)
        
    elif params['optimiser']=='sgd' and params['nesterov']=='False':
        opt = tf.keras.optimizers.SGD(learning_rate=lr_rate, momentum=params['momentum'], nesterov=False)

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=params['label_smoothing']),
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

## Augment data

In [11]:
def augment_data(directory, batch):
    '''Prepares train-time augmentation using given training and validations data)
    
    Returns train_data, val_data'''

    datagen_train = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=True,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=True,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
#             width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
#             height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            brightness_range=[0.6, 1.4],
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            )
    
    datagen_test = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)

    # Classes give the folders storing the two different categories
    train_data = datagen_train.flow_from_directory(directory + '/train',
                                             target_size=(params['target_size'], params['target_size']), batch_size = batch)
    
    val_data = datagen_test.flow_from_directory(directory + '/validation',
                                             target_size=(params['target_size'], params['target_size']), batch_size = batch)
    
    return train_data, val_data

In [ ]:
# Cell which previews data
t,v = augment_data('../all_faces_disk/home/jupyter/forensics_split', 8)
b=8
c=0
while True:
    fig, axe = plt.subplots(1,b, figsize=(28,10))
    for j,i in enumerate(t.next()[0]):
        axe[j].imshow(i, interpolation="nearest")
        axe[j].set_xticks([])
        axe[j].set_yticks([])
    plt.show()
    c+=1
    if c==4:
        break

## Auxiliary train time functions

In [12]:
def calculate_class_weights(train_data):
    '''Calculates class weights that weight the data based on the imbalance.
    Allows for better analysis in the case of imbalanced data - has no effect
    if data is balanced since the weights are then equal for each class.
    Use the generator obtained from the flow_from_directory method to obtain
    the class_weights.
    
    Input:
    train_data: the generator obtained during augmentation
    
    Returns a dictionary with class weights, required format for training'''
    
    # Calculate class weights which are required to fully balance the classes
    # Compares frequencies of appearence for each distinct label
    
    # The line of code below can be used on a generator to find the index labels
    print('Ensure class weights function corresponds to these class indices:',
          train_data.class_indices)
    
    counter = Counter(train_data.classes)                          
    max_val = float(max(counter.values()))       
    class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

    return class_weights

def load_model_weights(model, architecture):
    '''An alternative to training if there are already some generated weights
    
    Takes a built model (and its architecture type) and loads the weights
    with the highest validation accuracy.
    
    If there are no saved weights, a message is printed.'''

    path_to_weights = "../all_faces_bucket/trained_models/weights/{}".format(config_file)
    # get all the weights file names in a list
    if os.path.exists(path_to_weights):
        all_weights = sorted(os.listdir(path_to_weights + '/'))
    # If there is at least one file
        if len(all_weights) >= 1:
            # Use weights from highest val acc
            model.load_weights(path_to_weights + '/' + 'highest_val_acc.hdf5')
            
        else: # otherwise warn that no weights were loaded
            print("There are no weights stored. Training model from scratch:")   
    
    else: # otherwise warn that no weights were loaded
        print("There are no weights stored. Training model from scratch:")   
        
def save_model_from_best_weights(dropout, lr_rate, architecture, frozen_base):
    '''Takes the weights with the highest val accuracy and saves the corresponding model.'''
    model = build_model(dropout, lr_rate, architecture, frozen_base)
    load_model_weights(model, architecture)
    model.save('../all_faces_bucket/trained_models/saved_models/{}.h5'.format(config_file))    

## Training functions

In [13]:
def train_model(model, train_data, val_data, epochs, class_weights, architecture, lr_rate):
    '''Trains a provided model.
    Takes 6 arguments:
    
    1. model: a built model with an architecture specified in the build function
    2. train_data: augmented data obtained from the augment_data function
    3. val_data: validation data obtained from the augment_data function
    4. epochs -- number of epochs
    5. class weights -- a dictionary with weights (equal for balanced data so
    no negative impact)
    6. architecture: can choose vgg, xception, resnet50, mobilenet or efficientnet
    7. lr_rate: initial learning rate
    '''
    
    # Make a trained_models folder if it doesn't exist
    if not os.path.exists('../all_faces_bucket/trained_models'):
        os.makedirs('../all_faces_bucket/trained_models')
    
    # Make a weights folder if it doesn't exist
    if not os.path.exists('../all_faces_bucket/trained_models/weights'):
        os.makedirs('../all_faces_bucket/trained_models/weights')
        
    # Make a weights folder for the architecture if it doesn't exist
    if not os.path.exists('../all_faces_bucket/trained_models/weights/{}'.format(config_file)):
        os.makedirs('../all_faces_bucket/trained_models/weights/{}'.format(config_file))

    # Below saves on file - the weights with the highest validation accuracy
    filepath="../all_faces_bucket/trained_models/weights/{}/highest_val_acc.hdf5".format(config_file)
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', 
                                verbose=1, save_best_only=True, mode='max')
    
    # Make a folder to store training accuracies if it doesn't exist
    if not os.path.exists('../all_faces_bucket/trained_models/training_accuracies'):
        os.makedirs('../all_faces_bucket/trained_models/training_accuracies')
    
    # Callback to save training accuracies after each epoch
    csv_logger = CSVLogger('../all_faces_bucket/trained_models/training_accuracies/{}.csv'.format(config_file),
                           separator=',', append=True)
    
    # Stop after 3 epochs if val_accuracy doesn't improve
    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=params['patience'])
                          
    # Load previous weights from training if there are any
    load_model_weights(model, architecture)
    
    if params['learning_rate_type']=='cosine_decay':
        # Set learning rate config 
        sample_count = train_data.n # number of training samples
        epochs = epochs # total epochs - affects total steps (and hence speed of decay)
        warmup_epoch = params['warmup_epochs'] # number of warmup epochs
        batch_size = train_data.batch_size
        learning_rate_base = lr_rate
        total_steps = int(epochs * sample_count / batch_size)

        warmup_steps = int(warmup_epoch * sample_count / batch_size)

        warm_up_lr = hp.WarmUpCosineDecayScheduler(learning_rate_base=learning_rate_base,
                                            total_steps=total_steps,
                                            warmup_learning_rate=0.0,
                                            warmup_steps=warmup_steps,
                                            hold_base_rate_steps=2,
                                            verbose=0)

        cb = [plot_losses, checkpoint, csv_logger, es, warm_up_lr]
        
    elif params['learning_rate_type']=='constant':
        cb = [plot_losses, checkpoint, csv_logger, es]
        
    elif params['learning_rate_type']=='increasing':
        pass

    history = model.fit(train_data, epochs=epochs,
              steps_per_epoch = train_data.n//train_data.batch_size,
              validation_data = val_data, 
              validation_steps = val_data.n//val_data.batch_size,
              class_weight=class_weights,
              callbacks=cb,
              initial_epoch=0,                    # start training epoch - useful if continuing training
              verbose=1,
              max_queue_size=100,                # maximum size for the generator queue
              workers=16,                        # maximum number of processes to spin up when using process-based threading
              use_multiprocessing=False)
    
    # Make a saved models folder if it doesn't exist
    if not os.path.exists('../all_faces_bucket/trained_models/saved_models'):
        os.makedirs('../all_faces_bucket/trained_models/saved_models')
        
    model.save_weights('../all_faces_bucket/trained_models/weights/{}/lastepoch.hdf5'.format(config_file)) 
    model.save('../all_faces_bucket/trained_models/saved_models/{}.h5'.format(config_file))    

## Unifying Function

In [14]:
def run_training(dropout = 0.5, lr_rate = 0.0001, architecture = 'vgg', 
                 batch = 32, epochs = 50, frozen_base=True):

    '''Builds a model based on the specified architecture, augments training
    data (reserving a fraction for validation), then computes class weights to
    balance data and trains the model.
    
    Inputs:
    1. dropout  -- for the model
    2. lr_rate
    3. architecture -- a choice of vgg, resnet50, mobilenet, xception and efficientnet
    4. batch -- batch size
    5. epochs -- number of epochs
    '''

    # Build a model, augment data, get class_weights and train the model
    # Strategy scope allows us to leverage multiple GPUs
    strategy = tf.distribute.MirroredStrategy()
    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
    
    with strategy.scope(): # Allows for parallel GPUs
        model = build_model(dropout, lr_rate, architecture, frozen_base)
    train_data, val_data = augment_data('../all_faces_disk/home/jupyter/forensics_split', batch)
    class_weights = calculate_class_weights(train_data)
    if params['class_weights']=='True':
        trained_model = train_model(model, train_data, val_data, epochs, class_weights, architecture, lr_rate)
    else:
        trained_model = train_model(model, train_data, val_data, epochs, None, architecture, lr_rate)

## Run training 
Note: Make sure CPUs have enough memory for each batch eg. 1 core with 3.75GB RAM cant take batches larger than 32.  
8 CPUs with 30GB RAM typically works well for batches of 256.

In this model: 4 cores/8GB and T4 used - took approx 8 hours to train top dense layers and fine tune. 

Also note that while multiprocessing speeds up training, it interacts badly with Tensorflow and leads to deadlocks. To be on the safe side, set use_multiprocessing to False when training. 

In [ ]:
# Train dense layers with base frozen
run_training(dropout = params['dropout'], lr_rate = params['learning_rate'], 
             architecture = params['architecture'], batch = params['batch_size'],
             epochs = params['epochs'], frozen_base=True)

<Figure size 432x288 with 0 Axes>

In [ ]:
# Unfreeze all weights to fine-tine model
# NBNB this will use same learming rate type as specified in params unless you change it 
run_training(dropout = params['dropout'], lr_rate = params['learning_rate']/10, 
             architecture = params['architecture'], batch = params['batch_size'],
             epochs = params['epochs'], frozen_base=False)

In [ ]:
# Save model to bucket from highest val acc weights
# save_model_from_best_weights(dropout = 0.5, lr_rate = 0.0002, architecture = 'mobilenet', frozen_base=False)